In [1]:
__NAME = 'Heat 2'
# root_path = '/content/drive/MyDrive/Colab Notebooks/Code/Heat eqn'
# %tensorflow_version 1.x
# !pip install pyDOE

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import os
# os.chdir(root_path)
# !pwd

# Imports

In [4]:
import tensorflow as tf
print(tf.version)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

<module 'tensorflow._api.v1.version' from 'I:\\Users\\Ragav\\miniconda3\\envs\\juptenflowgpu115\\lib\\site-packages\\tensorflow_core\\_api\\v1\\version\\__init__.py'>
True
True


In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# from plotting import newfig, savefig
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FuncAnimation, PillowWriter

In [6]:
import numpy as np
import scipy.io
from scipy.interpolate import griddata
import time
from pyDOE import lhs

In [7]:
import pickle as pkl

In [8]:
%matplotlib widget

# Equation

In [9]:
k = 0.061644

$$
\frac{\partial u}{\partial t} = k \frac{\partial^2 u}{\partial x^2}
$$

# Load Data

# Model

In [10]:
# Layers
u_layers = [2, 50, 50, 50, 50, 1]
pde_layers = [3, 100, 100, 1]

layers = [2, 50, 50, 50, 50, 1]

In [11]:
# tf placeholders for Identification
t_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])
t_tf, x_tf, u_tf

(<tf.Tensor 'Placeholder:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_2:0' shape=(?, 1) dtype=float32>)

In [12]:
def initialize_NN(layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l + 1]])
        b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                        dtype=tf.float32)
        weights.append(W)
        biases.append(b)
    return weights, biases

def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim],
                                           stddev=xavier_stddev,
                                           dtype=tf.float32),
                       dtype=tf.float32)

In [13]:
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    H = X
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.sin(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [14]:
weights, biases = initialize_NN(layers)
# weights, biases

In [15]:
# load weights and biases

with open(__NAME + '/weights.pkl', 'rb') as db_file:
    W_pkl = pkl.load(db_file)

with open(__NAME + '/biases.pkl', 'rb') as db_file:
    B_pkl = pkl.load(db_file)

W = []
B = []
for w, b in zip(W_pkl, B_pkl):
    W.append(tf.Variable(w))
    B.append(tf.Variable(b))

weights = W
biases = B

In [16]:
lb_tf = tf.placeholder(tf.float32, shape=[2])
ub_tf = tf.placeholder(tf.float32, shape=[2])

In [17]:
# tf placeholders for Solution
t0_tf = tf.placeholder(tf.float32, shape=[None, 1])
x0_tf = tf.placeholder(tf.float32, shape=[None, 1])
u0_tf = tf.placeholder(tf.float32, shape=[None, 1])

t_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
t_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_x_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_x_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])

t_f_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_f_tf = tf.placeholder(tf.float32, shape=[None, 1])

In [18]:
def sol_net_u(t, x):
    X = tf.concat([t, x], 1)
    H = 2.0 * (X - lb_tf) / (ub_tf - lb_tf) - 1.0
    u = neural_net(H, weights, biases)
    u_x = tf.gradients(u, x)[0]
    return u, u_x

def sol_net_f(t, x):
    u, u_x = sol_net_u(t, x)

    u_t = tf.gradients(u, t)[0]

    u_xx = tf.gradients(u_x, x)[0]

    f = u_t - k * u_xx

    return f

In [19]:
# tf graphs for Solution
u0_pred, u_x0_pred = sol_net_u(t0_tf, x0_tf)
u_lb_pred, u_x_lb_pred = sol_net_u(t_lb_tf, x_lb_tf)
u_ub_pred, u_x_ub_pred = sol_net_u(t_ub_tf, x_ub_tf)
sol_f_pred = sol_net_f(t_f_tf, x_f_tf)

In [20]:
# loss for Solution
sol_loss =  tf.reduce_sum(tf.square(u0_tf - u0_pred)) + \
            tf.reduce_sum(tf.square(u_x_lb_tf - u_x_lb_pred)) + \
            tf.reduce_sum(tf.square(u_x_ub_tf - u_x_ub_pred)) + \
            tf.reduce_sum(tf.square(sol_f_pred))

In [21]:
# Optimizer for Solution
sol_optimizer = tf.contrib.opt.ScipyOptimizerInterface(
    sol_loss,
    var_list = weights + biases,
    method='L-BFGS-B',
    options={
        'maxiter': 50000,
        'maxfun': 50000,
        'maxcor': 50,
        'maxls': 50,
        'ftol': 1.0 * np.finfo(float).eps
    })

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [22]:
adam_optimizer = tf.train.AdamOptimizer()
sol_train_op_Adam = adam_optimizer.minimize(
            sol_loss,
            var_list= weights + biases)

In [23]:
# tf session
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



# Training

## Prepare data

In [24]:
lb = np.array([0.0, 0.0])
ub = np.array([2.0, 1.0])

In [25]:
N = 10
fig = plt.figure()
ax = fig.gca()
ax.set_xlim(lb[0], ub[0])
ax.set_ylim(lb[1], ub[1])
ax.set_xticks(np.arange(lb[0],ub[0],(ub[0] - lb[0])/N))
ax.set_yticks(np.arange(lb[1],ub[1],(ub[1] - lb[1])/N))
plt.grid()
l = lb + (ub - lb) * lhs(2, N) 
plt.scatter(l[:, 0], l[:, 1], color="r", label="lhs")
plt.title("Latin Hypercube Sampling\nN=10")
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
fig.set_figheight(3.8)
fig.set_figwidth(6)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
N0 = 200
N_b = 300
N_f = 20000
(N0, N_b, N_f)

(200, 300, 20000)

In [27]:
t_data = np.linspace(lb[0], ub[0], N_b)[:, None]
x_data = np.linspace(lb[1], ub[1], N0)[:, None]

In [28]:
T_sol, X_sol = np.meshgrid(t_data, x_data)
# U_sol = u_data

In [29]:
X_sol_star = np.hstack(
    (T_sol.flatten()[:, None],
    X_sol.flatten()[:, None])
    )
# U_sol_star = U_sol.flatten()[:, None]

print(X_sol_star.shape, X_sol_star[0:10], sep='\n')

(60000, 2)
[[0.         0.        ]
 [0.00668896 0.        ]
 [0.01337793 0.        ]
 [0.02006689 0.        ]
 [0.02675585 0.        ]
 [0.03344482 0.        ]
 [0.04013378 0.        ]
 [0.04682274 0.        ]
 [0.05351171 0.        ]
 [0.06020067 0.        ]]


In [30]:
L = 1

In [31]:
u_data = np.sin(np.pi * x_data / L)

In [32]:
X0 = np.hstack((T_sol[:, 0:1], X_sol[:, 0:1])) # left boundary
X_lb = np.hstack((T_sol[0:1, :].T, X_sol[0:1, :].T)) # lower boundary
X_ub = np.hstack((T_sol[0:1, :].T, np.repeat(ub[1], t_data.shape[0])[:, None])) # upper boundary

In [33]:
# shuffled initial boundary data (left boundary)
idx_x = np.random.choice(x_data.shape[0], N0, replace=False)
X0_train = X0[idx_x, :]
u0_train = u_data[idx_x, 0:1]

In [34]:
# shuffle time data
idx_t = np.random.choice(t_data.shape[0], N_b, replace=False)
tb_train = t_data[idx_t, :]

In [35]:
X_f_train = lb + (ub - lb) * lhs(2, N_f) 

In [36]:
fig, ax = plt.subplots()
ax.set_xlim(lb[0] -0.1, ub[0])
ax.set_ylim(lb[1] - 0.4, ub[1] + 0.4)
fig.set_figheight(3.2)
fig.set_figwidth(6)

ax.scatter(X0_train[:, 0], X0_train[:, 1], s=4, marker='.')
ax.scatter(tb_train[:, 0], np.repeat(lb[1], N_b), s=4, marker='.')
ax.scatter(tb_train[:, 0], np.repeat(ub[1], N_b), s=4, marker='.')
ax.scatter(X_f_train[:, 0], X_f_train[:, 1], s=4, marker='.', edgecolors='none')
# ax.imshow(u0_train, extent=(t_data[0, 0], t_data[1, 0], x0_train.max(), x0_train.min()), aspect='auto')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
fig, ax = plt.subplots()
ax.set_xlabel('$x$')
ax.set_ylabel('$u$')
ax.plot(x_data[:, 0], u_data[:, 0:1])
fig.set_figheight(3.2)
fig.set_figwidth(6)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
def callback(loss):
    print('Loss: %e' % (loss))

In [39]:
tf_dict = {
    lb_tf: lb,
    ub_tf: ub,
    t0_tf: X0_train[:, 0:1],
    x0_tf: X0_train[:, 1:2],
    u0_tf: u0_train,
    t_lb_tf: X_lb[:, 0:1],
    x_lb_tf: X_lb[:, 1:2],
    t_ub_tf: X_ub[:, 0:1],
    x_ub_tf: X_ub[:, 1:2],
    u_x_lb_tf: np.repeat(0, N_b)[:, None],
    u_x_ub_tf: np.repeat(0, N_b)[:, None],
    t_f_tf: X_f_train[:, 0:1],
    x_f_tf: X_f_train[:, 1:2]
}

In [40]:
fig, ax = plt.subplots()
ax.set_xlabel('$x$')
ax.set_ylabel('$u$')
ax.scatter(X0_train[:, 1:2], u0_train)
fig.set_figheight(3.2)
fig.set_figwidth(6)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
start_time = time.time()
it = 0
end = False
while not(end):

    sess.run(sol_train_op_Adam, tf_dict)

    # Print
    if it % 10 == 0:
        elapsed = time.time() - start_time
        loss_value = sess.run(sol_loss, tf_dict)
        print('It: %d, Loss: %.3e, Time: %.2f' %
                (it, loss_value, elapsed))
        start_time = time.time()

        if loss_value < 5 * 10**(-1):
            end = True

    it = it + 1

It: 0, Loss: 1.102e+03, Time: 2.95
It: 10, Loss: 5.190e+01, Time: 0.28
It: 20, Loss: 7.309e+01, Time: 0.27
It: 30, Loss: 1.274e+01, Time: 0.26
It: 40, Loss: 5.873e+00, Time: 0.27
It: 50, Loss: 4.257e+00, Time: 0.26
It: 60, Loss: 2.421e+00, Time: 0.26
It: 70, Loss: 1.312e+00, Time: 0.26
It: 80, Loss: 9.169e-01, Time: 0.26
It: 90, Loss: 7.219e-01, Time: 0.26
It: 100, Loss: 6.346e-01, Time: 0.26
It: 110, Loss: 5.742e-01, Time: 0.27
It: 120, Loss: 5.237e-01, Time: 0.26
It: 130, Loss: 4.869e-01, Time: 0.26


In [42]:
sol_optimizer.minimize(sess,
                       feed_dict=tf_dict,
                       fetches=[sol_loss],
                       loss_callback=callback)

Loss: 4.869232e-01
Loss: 1.365872e+05
Loss: 4.860751e-01
Loss: 4.855388e-01
Loss: 4.849139e-01
Loss: 4.835548e-01
Loss: 4.803027e-01
Loss: 4.724025e-01
Loss: 4.589138e-01
Loss: 4.365849e-01
Loss: 4.000503e-01
Loss: 3.601400e-01
Loss: 3.413003e-01
Loss: 3.341318e-01
Loss: 3.278770e-01
Loss: 3.250495e-01
Loss: 3.210161e-01
Loss: 3.154126e-01
Loss: 3.083426e-01
Loss: 3.093622e-01
Loss: 3.053403e-01
Loss: 3.017175e-01
Loss: 2.979198e-01
Loss: 2.935389e-01
Loss: 2.879314e-01
Loss: 2.845185e-01
Loss: 2.806233e-01
Loss: 2.784786e-01
Loss: 2.748318e-01
Loss: 2.690546e-01
Loss: 2.702209e-01
Loss: 2.668289e-01
Loss: 2.643198e-01
Loss: 2.629554e-01
Loss: 2.612739e-01
Loss: 2.584305e-01
Loss: 2.595767e-01
Loss: 2.571093e-01
Loss: 2.553497e-01
Loss: 2.539151e-01
Loss: 2.518907e-01
Loss: 2.482077e-01
Loss: 2.426942e-01
Loss: 2.398039e-01
Loss: 2.361719e-01
Loss: 2.347252e-01
Loss: 2.328631e-01
Loss: 2.286079e-01
Loss: 2.234474e-01
Loss: 2.195334e-01
Loss: 2.152082e-01
Loss: 2.117939e-01
Loss: 2.0449

In [43]:
sess.run(sol_loss, feed_dict=tf_dict)

0.07984073

In [44]:
with open(__NAME + '/weights.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(weights), file=db_file)

with open(__NAME + '/biases.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(biases), file=db_file)

In [45]:
u_pred = sess.run(u0_pred, {
        lb_tf: lb,
        ub_tf: ub,
        t0_tf: X_sol_star[:, 0:1],
        x0_tf: X_sol_star[:, 1:2]
    })

In [46]:
fig = plt.figure(figsize=(4*1.75,4), dpi=200)
ax = fig.gca()

ax.set_xlim(lb[0], ub[0])
ax.set_ylim(lb[1], ub[1])

# plt.subplots_adjust(bottom=0.17)
# plt.subplots_adjust(left=0.17)

plt.title('T')
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')

plt.pcolormesh(np.reshape(X_sol_star[:, 0], (N0, -1)), 
               np.reshape(X_sol_star[:, 1], (N0, -1)), 
               np.reshape(u_pred[:, 0], (N0, -1)), 
               shading='gouraud',  cmap='jet')
plt.colorbar()

plt.tight_layout()

# plt.legend()

fig.savefig('Figures\\Heat 2.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
t = np.reshape(X_sol_star[:, 0], (N0, -1))
x = np.reshape(X_sol_star[:, 1], (N0, -1))
u = np.reshape(u_pred[:, 0], (N0, -1))

x_init = x[:, 0]
u_init = u[:, 0]

fig = plt.figure(figsize=(4*1.75,4), dpi=200)
ax = fig.gca()

ax.set_xlim(lb[1], ub[1])

ax.yaxis.grid(color='gainsboro', linestyle='dotted', linewidth=1.5)
ax.xaxis.grid(color='gainsboro', linestyle='dotted', linewidth=0.8)
ax.axhline(0,linestyle='dotted', color='grey')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

plt.title('t = 0')

ax.set_xlabel('$x$')
ax.set_ylabel('$T$')

plt.tight_layout()

ln, = ax.plot(x_init, u_init)

def update(frame):
    plt.title('t = {time:.2f}'.format(time = t[0, frame]))
    ln.set_data(x[:, frame], u[:, frame])

ani = FuncAnimation(fig, update, list(range(0, N_b)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
ani.event_source.stop()

In [60]:
# anim.event_source.stop()
writer = PillowWriter(fps=25) 
ani.save("Figures\\Heat 2.gif", writer=writer)
# ani.event_source.stop()